## **Meflah wided _22214601**

# Notebook GAN

Dans ce notebook, nous allons mis en place un GAN dans le but de générer des images

# **NB :** Par erreur, j'ai perdu les résultats. Cependant, j'avais déjà conservé ce notebook avec les résultats au format PDF précédemment. Je joins à ce notebook sa version PDF complète. Je m'excuse pour cela.

## Installation

In [1]:
# utiliser cette cellule pour installer les librairies manquantes
# pour cela il suffit de taper dans cette cellule : !pip install nom_librairie_manquante
# d'exécuter la cellule et de relancer la cellule suivante pour voir si tout se passe bien
# recommencer tant que toutes les librairies ne sont pas installées ...

In [2]:
# Importation des différentes librairies utiles pour le notebook

# Tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle
import pandas as pd
#from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys
import copy
import random
from numpy import mean
from numpy import vstack
from numpy import std
from numpy import ones
from numpy import zeros
from numpy.random import randint


# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import plotly.graph_objs as go
import plotly.offline as py

from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# TensorFlow et keras
import tensorflow as tf
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array, load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from keras.models import load_model
from sklearn.model_selection import KFold, StratifiedKFold
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import os
from os import listdir
from os.path import isfile, join
import cv2
import glob
from numpy import expand_dims
import matplotlib.pyplot as plt

In [3]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
import sys
my_local_drive='/content/gdrive/MyDrive/ML2_prjt2023'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

In [ ]:
IMG_SIZE=124
COLUMNS = 25 # Nombre d'images à afficher

#Donne une liste d'images à la bonne dimension avec leur classe
def create_training_data(path_data, list_classes):
  training_data=[]
  for classes in list_classes:
      path=os.path.join(path_data, classes)
      class_num=list_classes.index(classes)
      for img in os.listdir(path):
        try:
          img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_UNCHANGED)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
          #new_array = cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB)
          training_data.append([new_array, class_num])
        except Exception as e:
          pass
  return training_data

def create_X_y (path_data, list_classes):
  # récupération des données
  training_data=create_training_data(path_data, list_classes)
  # tri des données
  random.shuffle(training_data)
  # création de X et y
  X=[]
  y=[]
  for features, label in training_data:
    X.append(features)
    y.append(label)
  X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
  y=np.array(y)
  return X,y


# Chargement des données
def load_data():
  my_path="Data_Project/Tiger-Fox-Elephant/"
  #my_path="./"
  my_classes=['fox']
  X,y=create_X_y (my_path,my_classes)

  # Ajout d'une dimension pour le canal (1) !! pas besoin de l'ajouter il y est déjà
  #Xi = expand_dims(X_train, axis=-1)
  # Normalisation
  Xi = X.astype('float32')
  # scale from [0,255] to [0,1]
  Xi = Xi / 255.0
  return Xi

def plot_examples(X):
  plt.figure(figsize=(60,60))
  for i in range(COLUMNS):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # cv2 lit met les images en BGR et matplotlib lit du RGB
    # X[i] = cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB)
    plt.imshow(X[i, :, :])

In [ ]:
def create_generator(latent_dim=100):
  model = Sequential()
    # 1/4 de l'image (7x7) * nombre d'images possibles
  n_nodes = 100 * 31 * 31
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((31, 31, 100)))
  # upsample to 14x14
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  # upsample to 28x28
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(3, (7,7), activation='sigmoid', padding='same'))
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

La définition de la partie du générateur est :        

In [ ]:
generator_model=create_generator()
generator_model.summary()

Nous pouvons tester le générateur pour voir un exemple d'image générée :     

In [ ]:
latent_dim=100
generator = create_generator()
noise=tf.random.normal([1,latent_dim])
generated_image = generator (noise, training=False)
plt.imshow (generated_image[0, :, :, 0])


In [ ]:
def create_discriminator():
  model_fox = Sequential()
  # Convolution and pooling
  model_fox.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), name="Conv2D_1", padding="same", strides=1))
  model_fox.add(tf.keras.layers.BatchNormalization(axis=-1))
  model_fox.add(MaxPooling2D(pool_size=(2, 2), name="Maxpooling2D_1"))
  model_fox.add(Dropout(0.3))

  # Convolution and pooling
  #model_fox.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3), name="Conv2D_2", padding="same", strides=1))
  #model_fox.add(tf.keras.layers.BatchNormalization(axis=-1))
  #model_fox.add(MaxPooling2D(pool_size=(2, 2), name="Maxpooling2D_2"))
  #model_fox.add(Dropout(0.3))

  # flatten
  model_fox.add(Flatten(name="flatten") )

  # Partie classification
  model_fox.add(Dense(512, activation='relu'))
  model_fox.add(Dense(1, activation='sigmoid'))

  #model_fox.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  model_fox.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy',metrics=['accuracy'])

  return model_fox

# def create_discriminator():
#   model = Sequential()
#   model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=[124,124,3]))
#   model.add(LeakyReLU(alpha=0.2))
#   model.add(Dropout(0.3))
#   model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
#   model.add(LeakyReLU(alpha=0.2))
#   model.add(Dropout(0.3))
#   model.add(Flatten())
#   model.add(Dense(1, activation='sigmoid'))
#   opt = Adam(learning_rate=0.0002, beta_1=0.5)
#   model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#   return model

Le modèle pour le discriminateur est :       

In [ ]:
model_discriminator=create_discriminator()
model_discriminator.summary()

In [ ]:
discriminator=create_discriminator()
prediction=discriminator(generated_image)
print (prediction)

In [ ]:
# Creation d'un jeu de données de vraies images
# les vraies images sont labélisées avec 1
def generate_real_samples(dataset, nb_images):
	# tirage aléatoire
	xi = randint(0, dataset.shape[0], nb_images)
	# sélection des images
	X = dataset[xi]
	# mettre 1 comme label de classe
	y = ones((nb_images, 1))
	return X, y

# Création d'un faux jeu de données
# elles seront labélisées avec 0
def generate_fake_samples(nb_images):
  # generation de nombres aléatoires entre 0 et 1
  # attention ici on simplifie en mettant la taille de l'image
  # normalement il faut prendre latent_dim
  X = np.random.rand(124 * 124 * nb_images * 3)
  # reshape en images grises (couleur !!!)
  X = X.reshape((nb_images, 124, 124, 3))
  # mettre 0 comme label de classe
  y = zeros((nb_images, 1))
  return X, y

In [ ]:
# train the discriminator model
def train_discriminator(model, dataset, epochs=100, batchsize=256):
  # on constitue un jeu de données de batchsize/2 images réelles et images fausses
	half_batch = int(batchsize / 2)
	# boucler sur les epochs
	for i in range(epochs):
		# sélection d'images réelles
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# mettre à jour le discriminateur avec les images réelles
		_, real_acc = model.train_on_batch(X_real, y_real)
		# generation de fausses images
		X_fake, y_fake = generate_fake_samples(half_batch)
		# mise à jour du discriminateur avec de fausses images
		_, fake_acc = model.train_on_batch(X_fake, y_fake)
		# Affichage des résultats pour l'accuracy des vraies et des fausses
		print('>%d accuracy_real=%.0f%% accuracy_fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [ ]:
%pwd

nb_images_total=256 # 128 images de chaque classe
epochs=50

# chargement du modèle
model = create_discriminator()

# chargement des données
dataset=load_data()

# lancement de l'entrainement du discriminateur
train_discriminator(model, dataset, epochs, nb_images_total)

In [ ]:
def create_gan(generator_model, discriminator_model):
  # mettre les poids du discriminateur non entrable
  discriminator_model.trainable = False
  # un seul modele qui regroupe
  model = Sequential()
  # ajout du générateur
  model.add(generator_model)
  # ajout du discriminateur
  model.add(discriminator_model)

  opt = Adam(learning_rate=0.0003, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

Le modèle GAN final ressemble à ceci :

In [ ]:
latent_dim = 100

# creation du générateur
generator_model = create_generator(latent_dim)
# creation du discriminateur
discriminator_model = create_discriminator()

# creation du gan
gan_model = create_gan(generator_model, discriminator_model)

gan_model.summary()



In [ ]:
def generate_latent_points (latent_dim, nb_images):
  X_input = np.random.randn(latent_dim * nb_images)
  X_input = X_input.reshape(nb_images, latent_dim)
  return X_input

def generate_fake_samples_for_generator(generator_model, latent_dim, nb_images):
  # generation des points
  X_input = generate_latent_points (latent_dim,nb_images)
  # prediction de la sortie du générateur
  X = generator_model.predict(X_input)
  # A ce niveau on considère que les images sont fausses
  # donc on met 0 comme label.
  y = zeros((nb_images, 1))
  return X, y

def plot_and_save_generatedimages(generated_images, epoch, nb_images=10):
	# Affichage des images
	for i in range(nb_images * nb_images):
		plt.subplot(nb_images, nb_images, 1 + i)
		plt.axis('off')
		plt.imshow(generated_images[i, :, :])
	# sauvegarde de l'image
	filename = 'generated_plot_Fox_withGan%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()



def evaluate_model (dataset, epoch, generator_model, discriminator_model, latent_dim, nb_images=100, save_model=True):
  # récupération de vraies images pour le discriminateur
  X_real, y_real = generate_real_samples(dataset, nb_images)
  # Evaluation de l'accuracy pour le discriminateur
  _, acc_real = discriminator_model.evaluate(X_real, y_real, verbose=0)

  # génération de fausses images pour le générateur et donc le gan
  X_fake, y_fake = generate_fake_samples_for_generator(generator_model, latent_dim, nb_images)
  # Evaluation du discriminateur avec des fausses images
  _, acc_fake = discriminator_model.evaluate(X_fake, y_fake, verbose=0)

  print ('Accuracy reélle : %.0f%%, fausse : %.0f%%' % (acc_real*100, acc_fake*100))

  plot_and_save_generatedimages(X_fake, epoch)
  if save_model==True:
    # sauvegarde du generateur pour un autre usage
    filename = 'generator_model_Fox%03d.h5' % (epoch + 1)
    generator_model.save(filename)




In [ ]:
# train the generator and discriminator
def train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs=100, batchsize=256):
  print("Dataset shape : ", dataset.shape)
  # Pour déterminer combien il y aura de batchs analysés à chaque epoch
  batches_per_epoch = int(dataset.shape[0] / batchsize)
  print("Batches_per_epoch : ", batches_per_epoch)

  # checkpoint_generator = tf.train.Checkpoint(generator_model)
  # checkpoint_discriminator = tf.train.Checkpoint(discriminator_model)
  # checkpoint_gan = tf.train.Checkpoint(gan_model)

  # pour récupérer le même nombre d'images fausses et vraies
  half_batch = int(batchsize / 2)

  for i in range(epochs):
    # parcours des batches pour chaque pas d'epoch
    for j in range(batches_per_epoch):
      # PARTIE DISCRIMINATOR
      #tirage aléatoire de half_batch images vraies
      X_real, y_real = generate_real_samples(dataset, half_batch)
      # generation de half_size fausses images
      X_fake, y_fake = generate_fake_samples_for_generator(generator_model, latent_dim, half_batch)
      # vstack permet de concaténer les vraies et fausses images dans X (resp. dans y)
      # on aurait pu utiliser np.concatenate (np.concatenate([X_real, X_fake]))
      X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

      # la ligne suivante permet au discriminateur de mettre à jour ses poids
      # attention dans le GAN il ne peut pas aprendre donc il ne peut le faire qu'ici
      discriminator_loss,_  = discriminator_model.train_on_batch(X, y)

      # PARTIE GENERATOR ET DONC DE L'ENTREE DU GAN
      # Création des points comme entrée du générateur
      #
      X_for_gan = generate_latent_points(latent_dim, batchsize)
      # On met un 1 pour les labels des fausses images pour faire croire au discriminateur
      # qu'il s'agit de vraies images
      y_for_gan = ones((batchsize, 1))

      # mise à jour des poids du générateur par propagation de l'esseur
      # du discriminateur
      gan_loss = gan_model.train_on_batch(X_for_gan, y_for_gan)
			# Affichage des loss
      print('epoch %d - batch %d/%d, discriminator_loss=%.3f, generator_loss=%.3f'%(i+1, j+1, batches_per_epoch, discriminator_loss, gan_loss))

    # Toutes les 20 epochs evaluation du modèle et sauvegarde du générateur
    if i==1 or i % 20==0:
        evaluate_model (dataset, i, generator_model, discriminator_model, latent_dim, save_model=False)

    if i==1 or i % 250==0:
        #Save checkpoints every 20 iterations
        evaluate_model (dataset, i, generator_model, discriminator_model, latent_dim)
        # save_path = checkpoint_generator.save('/training_checkpoints_gan/generator')
        # save_path = checkpoint_discriminator.save('/training_checkpoints_gan/discriminator')
        # save_path = checkpoint_gan.save('/training_checkpoints_gan/gan')

In [ ]:
latent_dim = 100
epochs=1000
batchsize=128
# Creation du discriminateur
discrimator_model = create_discriminator()
# Creation du generateur
generator_model = create_generator(latent_dim)
# Creation du GAN
gan_model = create_gan(generator_model, discriminator_model)
# Chargement des données
dataset = load_data()


In [ ]:
# train model
# Avec batchsize 25, epochs 1000
# model = train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs,batchsize)
#print(dataset.shape)

In [ ]:
# train model
# Avec batchsize 10, epochs 1000
# model = train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs,batchsize)
#print(dataset.shape)

In [ ]:
# train model
# Avec batchsize 512, epochs 1000
epochs=2500
batchsize=32
model = train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs,batchsize)
#print(dataset.shape)

In [ ]:
epochs=3000
batchsize=10
model = train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs,batchsize)

In [ ]:
#model = load_model('modeleFoxAmelioreIDG.h5')

In [ ]:
# Création d'un vecteur aléatoire de taille latent_dim
latent_points = generate_latent_points(100, 1)

In [ ]:
latent_points = generate_latent_points(100, 1)
generated_image = generator_model.predict(latent_points)
print(generated_image.shape)
plt.imshow(generated_image[0, :, :])

In [ ]:
plt.imshow (generated_image[0, :, :, 0], cmap=plt.cm.binary)